## What ot keep
- Dept
- Call_reason
- Source_id
- Council_district
- resolution_days_due

## What to drop ( will be dropping inprep durig second iteration)
- case_status
- case_id
- address
- zipcode
- days_open
- closed day
- is_late
- days_before_or_after_due
- due_date
- open_date
- latitude
- longitude

In [1]:
#imports to be able to pull data and prepare it for us
import wrangle
import model

#imports for needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing

# Acquire

In [2]:
df = pd.read_csv('service_calls.csv')

# Prepare

In [3]:
df = wrangle.clean_311(df)

In [4]:
df = model.model_df()

In [5]:
df = df.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district'])

In [6]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = model.split_separate_scale(df)

In [7]:
train.head()

,resolution_days_due,level_of_delay,district_0,district_1,district_2,district_3,district_4,district_5,district_6,district_7,...,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests
CASEID,,,,,,,,,,,,,,,,,,,,,
1016517770,2,Early Response,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
1016203792,9,Early Response,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1016771848,3,Early Response,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1016634860,6,Early Response,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1016302671,1,On Time Response,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Baseline

In [8]:
y_train.value_counts()

Early Response              148198
On Time Response             28857
Very Early Response          15299
Late Response                 5005
Very Late Response            1763
Extremely Late Response        555
Extremely Early Response        83
Name: level_of_delay, dtype: int64

In [9]:
baseline_accuracy = (train.level_of_delay == 'Early Response').mean()
print(f'My baseline accuracy is: \n', round(baseline_accuracy, 3))

My baseline accuracy is: 
 0.742


# SGD Classifier

In [10]:
X_train.head()

,resolution_days_due,district_0,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,...,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests
CASEID,,,,,,,,,,,,,,,,,,,,,
1016517770,2,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1016203792,9,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1016771848,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1016634860,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1016302671,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Vanilla SGD Classifier

In [20]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123)

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.81
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.81


### l1 SGD Classifier

In [23]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, penalty='l1')

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.8
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.8


### Optimal Learning SGD Classifier

In [24]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, learning_rate='optimal')

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.81
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.81


### Balanced SGD Classifier

In [25]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, class_weight='balanced')

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.6
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.61


### Elasticnet SGD Classifier

In [26]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, penalty='elasticnet')

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.68
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.68


## Hinge SGD Classifier

In [30]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, loss="hinge")

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.81
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.81


In [32]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, learning_rate='optimal', loss="hinge", penalty='l1')

clf3.fit(X_train, y_train)

y_pred = clf3.predict(X_train)

print('Accuracy of SGD classifier on training set:', round(clf3.score(X_train, y_train),2))
print('________________________________________________________________')
print('Accuracy of SGD classifier on validate set:', round(clf3.score(X_validate, y_validate),2))

Accuracy of SGD classifier on training set: 0.8
________________________________________________________________
Accuracy of SGD classifier on validate set: 0.8
